# Úkol 2

Tentokrát budeme pracovat s daty z výzkumu veřejného mínění Eurobarometr. Ten je realizován v cca 40 zemích a lidé jsou v něm pravidelně tázání na řadu otázek. My se podíváme na tři z nich.

Přehled států EU, které jsou v průzkumu, najdeš v souboru [countries.csv](soubory_02/countries.csv). V první části úkolu můžeš pracovat se všemi státy nebo jen pro státy EU - záleží na tobě. Ve druhé a třetí části pracuj pouze se státy EU.

In [1]:
import pandas as pd # ano
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
data_country = pd.read_csv("countries.txt") # tento DF používám pro celý Ukol_2
data_country.head()
# data_country.info()

,Country Name,Country,Euro
0,Belgium,BE,1
1,Greece,EL,1
2,Lithuania,LT,1
3,Portugal,PT,1
4,Bulgaria,BG,0


## Inflace

V souboru [ukol_02_a.csv](soubory_02/ukol_02_a.csv) najdeš procenta lidí, kteří považují inflaci a růst životních nákladů za jeden ze svých nejzávažnějších problémů. Data jsou za dvě období - léto 2022 (sloupec 97) a zima 2022/2023 (sloupec 98). Ověř, zda se procento lidí, kteří řadí inflaci mezi 2 své nejzávažnější problémy, změnilo.

Je vhodné provést následující postup:

- Test normality obou skupin dat. Podle toho zjistíš, zda je lepší provést parametrický nebo neparametrický test.
- Formulace hypotéz testu.
- Výběr vhodného testu. Vhodný je test, který jsme na lekci nepoužívali, ale je v seznamu testů, který je součástí lekce 6. Důležité je uvědomit si, že **porovnáváme tu samou skupinu států ve dvou různých časových obdobích**.
- Formulace výsledek testu (na základě p-hodnoty).

načtení dat/úprava dat (pro mě)

In [3]:
data_inflace = pd.read_csv("ukol_02_a.txt")
# data_inflace.head()
# data_inflace.info()
data_inflace.rename(columns={'98':'winter_23', '97':'summer_22'}, inplace = True) # aych věděla co je, který sloupec
data_inflace.head()

,Country,winter_23,summer_22
0,BE,31,34
1,BG,22,31
2,CZ,34,42
3,DK,28,33
4,D-W,33,40


- Test normality obou skupin dat - podle výsledku vyberu/provedu parametrický nebo neparametrický test.

Hypotéza:


H0: winter_23 resp. summer_22 mají normální rozdělení dat (p<0,05 zamítám H0 t.j. platí H1.....p>0,05 nezamítám H0)

H1: winter_23 resp. summer_22 nemají normální rozdělení dat 

Shapiro-Wilk test

In [4]:
res_winter_23 = stats.shapiro(data_inflace['winter_23']) # ověření normality dat 
res_winter_23 # statistic=0.9803104996681213, pvalue=0.687289297580719 p>0,05 - normální rozdělení dat

ShapiroResult(statistic=0.9803104996681213, pvalue=0.687289297580719)

In [5]:
res_summer_22 = stats.shapiro(data_inflace['summer_22']) # ověření normality dat 
res_summer_22 # statistic=0.9694532752037048, pvalue=0.33090925216674805 p>0,05 - normální rozdělení dat

ShapiroResult(statistic=0.9694532752037048, pvalue=0.33090925216674805)

Mohu použít parametrické testy

- Formulace hypotéz testu.

Stejne zeme, stejna otazka dve pozorovani. Normalni rozdeleni dat.

Ověř, zda se procento lidí, kteří řadí inflaci mezi 2 své nejzávažnější problémy, změnilo. summer_22/winter_23

H0 - procento lidi je stejné (t.j. nezmněnilo se) takže pokud p< 0,05 zamítám H0 a H1 platí

H1 - procento lidí není stejné (t.j. zmněnilo se)

test: párový t-test  scipy.stats.ttest_rel (a, b, axis=0, nan_policy='propagate', alternative='two-sided', *, keepdims=False)
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html#scipy.stats.ttest_rel

- Výběr vhodného testu.

In [8]:
res_inflace = stats.ttest_rel(data_inflace['summer_22'], data_inflace['winter_23'])
res_inflace  # TtestResult(statistic=3.868878598419143, pvalue=0.0003938172257904746, df=40)

TtestResult(statistic=3.868878598419143, pvalue=0.0003938172257904746, df=40)

- Formulace výsledek testu (na základě p-hodnoty).

pvalue=0.0003938172257904746 .... p<0,05  zamítám H0 tzn. Ověřila jsem, že procento lidí, kteří řadí inflaci mezi 2 své nejzávažnější problémy, se změnilo.

## Důvěra ve stát a v EU

Ve výzkumu je dále zkoumáno, jak moc lidé věří své národní vládě a jak moc věří EU. Data jsou v souboru [ukol_02_b.csv](soubory_02/ukol_02_b.csv). Číslo udává procento lidí, kteří dané instituci věří. Ověř, zda existuje korelace mezi procentem lidí, které věří EU a procentem lidí, kteří věří své národní vládě.

Je vhodné provést následující postup:

- Test normality obou skupin dat.
- Formulace hypotéz testu.
- Volba vhodného testu. Pokud data nemají normální rozdělení, můžeš využít test korelace, který jsme prováděli na lekci. Pokud data normální rozdělení mají, můžeš použít test zmíněný v přehledu testů v dané lekci.

In [9]:
data_duvera_stat_EU = pd.read_csv("ukol_02_b.txt")
#data_duvera_stat_EU.info()
data_duvera_stat_EU.head()

,Country,National Government Trust,EU Trust
0,BE,40,55
1,BG,19,49
2,CZ,30,43
3,DK,52,65
4,D-W,54,53


In [11]:
data_duvera_stat_EU_merge = pd.merge(data_duvera_stat_EU, data_country , on=['Country']) # jen státy EU
data_duvera_stat_EU_merge.head() # s tímto DF budu dále pracovat
# data_duvera_stat_EU_merge.info()

,Country,National Government Trust,EU Trust,Country Name,Euro
0,BE,40,55,Belgium,1
1,BG,19,49,Bulgaria,0
2,CZ,30,43,Czechia,0
3,DK,52,65,Denmark,0
4,DE,49,49,Germany,1


- Test normality obou skupin dat.

Hypotéza:


H0: National Government Trust resp. EU Trust mají normální rozdělení dat (p<0,05 zamítám H0 t.j. platí H1.....p>0,05 nezamítám H0)

H1: National Government Trust resp. EU Trust nemají normální rozdělení dat 

Shapiro-Wilk test

In [12]:
res_National_Government_Trust = stats.shapiro(data_duvera_stat_EU_merge['National Government Trust']) # ověření normality dat 
res_National_Government_Trust # ShapiroResult(statistic=0.9438267350196838, pvalue=0.15140558779239655) p>0,05 - nezamítám H0 - normální rozdělení dat

ShapiroResult(statistic=0.9438267350196838, pvalue=0.15140558779239655)

In [13]:
res_EU_Trust = stats.shapiro(data_duvera_stat_EU_merge['EU Trust']) # ověření normality dat 
res_EU_Trust # ShapiroResult(statistic=0.9735807180404663, pvalue=0.6981646418571472) p>0,05 - nezamítám H0 - normální rozdělení dat

ShapiroResult(statistic=0.9735807180404663, pvalue=0.6981646418571472)

- Formulace hypotéz testu.

Ověř, zda existuje korelace mezi procentem lidí, které věří EU a procentem lidí, kteří věří své národní vládě.

H0: procento lidí, které věří EU, a procento lidí, kteří věří své národní vládě - nejsou statisticky závislé - takže pokud p< 0,05 zamítám H0 a H1 platí

H1: procento lidí, které věří EU, a procento lidí, kteří věří své národní vládě - jsou statisticky závislé

- Volba vhodného testu.

1. Normální rozdělení dat
2. Test statistické významnosti korelace

test založený na Pearsonově korelačním koeficientu 

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr


In [14]:
res_duvera_EU = stats.pearsonr(data_duvera_stat_EU_merge['EU Trust'], data_duvera_stat_EU_merge['National Government Trust'])
res_duvera_EU # PearsonRResult(statistic=0.6097186340024556, pvalue=0.0007345896228823406) p<0,05 zamítám H0 a platí H1

PearsonRResult(statistic=0.6097186340024556, pvalue=0.0007345896228823406)

## Důvěra ve stát e euro

Nakonec si rozdělíme státy EU na dvě skupiny - státy v eurozóně a státy mimo ni. Jak je to s důvěrou v EU? Důvěřují EU více lidé, kteří žijí ve státech platící eurem? Využij znovu data o důvěře v EU ze souboru [ukol_02_b.csv](soubory_02/ukol_02_b.csv) a rozděl státy na ty, které jsou v eurozóně, a ty, které jsou mimo ni. Porovnej, jak se liší důvěra v EU v těchto dvou skupinách zemí. Státy můžeš rozdělit s využitím tabulky v souboru [countries.csv](countries.csv).

- Test normality můžeš vynechat, řiď se výsledkem z předchozí části.
- Formulace hypotéz testu.
- Volba vhodného testu. Pokud data nemají normální rozdělení, můžeš využít test z bonusového úkolu ze 7 lekce. Pokud data normální rozdělení mají, můžeš použít test zmíněný v přehledu testů v dané lekci.

Zdroj dat: [https://europa.eu/eurobarometer/screen/home](https://europa.eu/eurobarometer/screen/home)

data_duvera_stat_EU_merge - DF, se kterým budu pracovat

- státy v eurozóně - staty_euro_yes
- státy mimo eurozónu - staty_euro_no

In [18]:
staty_euro_yes =data_duvera_stat_EU_merge[data_duvera_stat_EU_merge["Euro"] == 1] 
#staty_euro_yes.info()
staty_euro_yes.head()

,Country,National Government Trust,EU Trust,Country Name,Euro
0,BE,40,55,Belgium,1
4,DE,49,49,Germany,1
6,IE,46,58,Ireland,1
7,EL,22,37,Greece,1
8,ES,23,50,Spain,1


In [20]:
staty_euro_no =data_duvera_stat_EU_merge[data_duvera_stat_EU_merge["Euro"] == 0] 
#staty_euro_no.info()
staty_euro_no.head()

,Country,National Government Trust,EU Trust,Country Name,Euro
1,BG,19,49,Bulgaria,0
2,CZ,30,43,Czechia,0
3,DK,52,65,Denmark,0
5,EE,39,48,Estonia,0
10,HR,20,42,Croatia,0


- Test normality můžeš vynechat, řiď se výsledkem z předchozí části. res_EU_Trust - pvalue=0.3959057033061981  p>0,05 - nezamítám H0 - normální rozdělení dat


Výběr testu provedeme na základě následujících informací:

- máme dva soubory,
- data nejsou párová (jsou to dvě různé skupiny států).
- chceme porovnat výši jejich důvěry v EU,

- Formulace hypotéz testu - Jak je to s důvěrou v EU? Důvěřují EU více lidé, kteří žijí ve státech platící eurem?

H0: Důvěra v EU ve státech v eurozóně - staty_euro_yes je stejná jako ve státech mimo eurozónu - staty_euro_no 

H1: Důvěra v EU ve státech v eurozóně - staty_euro_yes a ve státech mimo eurozónu - staty_euro_no je různá

- Volba vhodného testu - data mají normální rozdělení 

Pro test hypotézy můžeme použít t-test. 
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html#scipy.stats.ttest_ind


In [21]:
res_porovnani_duvera_EU = stats.ttest_ind(staty_euro_yes["EU Trust"], staty_euro_no["EU Trust"])
res_porovnani_duvera_EU # Ttest_indResult(statistic=-0.33471431258258433, pvalue=0.7406326832748829) 

Ttest_indResult(statistic=-0.33471431258258433, pvalue=0.7406326832748829)